In [ ]:
# Workflow

In [1]:
import arcpy as ARCPY
import arcgisscripting as ARC
import SSDataObject as SSDO
import SSUtilities as UTILS
import WeightsUtilities as WU
import numpy as NUM
import scipy as SCIPY
import pysal as PYSAL
import os as OS
import pandas as PANDAS

In [2]:
inputFC = r'../data/CA_Polygons.shp'
fullFC = OS.path.abspath(inputFC)
fullPath, fcName = OS.path.split(fullFC)
ssdo = SSDO.SSDataObject(inputFC)
uniqueIDField = "MYID"
fieldNames = ['GROWTH', 'LOGPCR69', 'PERCNOHS', 'POP1969']
ssdo.obtainData(uniqueIDField, fieldNames)
df = ssdo.getDataFrame()
print(df.head())

       GROWTH  LOGPCR69  PERCNOHS  POP1969
158  0.011426  0.176233      37.0  1060099
159 -0.137376  0.214186      38.3      398
160 -0.188417  0.067722      41.4    11240
161 -0.085070 -0.118248      42.9   101057
162 -0.049022 -0.081377      48.1    13328


In [3]:
import pysal2ArcUtils as PYSAL_UTILS
swmFile = OS.path.join(fullPath, "rook_bin.swm")
W = PYSAL_UTILS.PAT_W(ssdo, swmFile)
w = W.w
X = df.as_matrix()
maxp = PYSAL.region.Maxp(w, X[:,0:3], 3000000., floor_variable = X[:,3])
maxp.regions
maxpGroups = NUM.empty((ssdo.numObs,), int)
for regionID, orderIDs in enumerate(maxp.regions):
    maxpGroups[orderIDs] = regionID
    print((regionID, orderIDs))

(0, [29, 18, 36])
(1, [56, 27, 33, 5, 48, 16, 22, 38, 4, 54, 49, 2, 47, 10, 52, 3, 50, 51, 57, 30, 1, 11, 44, 31, 45, 24, 8, 17, 6, 7, 46, 28, 20, 25, 13])
(2, [55, 41, 14, 53, 15, 9, 19, 21, 39, 35, 32, 12, 34, 23])
(3, [37, 40, 42, 0, 43, 26])


In [4]:
swmFile = OS.path.join(fullPath, "queen.swm")
W = PYSAL_UTILS.PAT_W(ssdo, swmFile)
w = W.w

kernelSWMFile = OS.path.join(fullPath, "knn8.swm")
KW = PYSAL_UTILS.PAT_W(ssdo, kernelSWMFile)
kw = KW.w

In [5]:
import AutoModel as AUTO
auto = AUTO.AutoSpace_PySAL(ssdo, "GROWTH", ['LOGPCR69', 'PERCNOHS', 'POP1969'],
                            W, KW, pValue = 0.1, useCombo = True)
ARCPY.env.overwriteOutput = True
outputFC = r'E:\Data\Conferences\esri_stat_summit_16\PYDemo\PYDemo.gdb\pysal_automodel'
auto.createOutput(outputFC)

In [6]:
ARCPY.env.overwriteOutput = True
outputFC = r'E:\Data\Conferences\esri_stat_summit_16\PYDemo\PYDemo.gdb\pysal_maxp'
outMax = SSDO.CandidateField('MAXP', 'LONG', maxpGroups + 1)
outFields = {'MAXP': outMax}
appendFields = fieldNames + ["NEW_NAME"]
ssdo.output2NewFC(outputFC, outFields, appendFields = appendFields)

In [7]:
import WeightConvertor as W_CONVERT
galFile = OS.path.join(fullPath, "queen.gal")
convert = W_CONVERT.WeightConvertor(swmFile, galFile, inputFC, "MYID", "SWM", "GAL")
convert.createOutput()